# EDS Block Fixes Testing Notebook

This notebook helps validate that EDS blocks follow standards after fixes are applied.

## What This Tests

1. **Reserved Class Names** - Verifies blocks don't use `.{blockname}-container` or `.{blockname}-wrapper`
2. **Block Parameter Scoping** - Ensures blocks use `block` parameter, not `document.querySelector()`
3. **No Inline CSS** - Confirms blocks use CSS classes instead of `element.style`
4. **CONFIG Objects** - Checks blocks have proper CONFIG object with centralized configuration
5. **Error Handling** - Validates async functions have try/catch blocks

## How to Use

1. **Run Cell 1** - Initialize the testing environment (ALWAYS RUN FIRST)
2. **Run verification cells** - Test specific blocks you've fixed
3. **Review results** - Green ✅ = pass, Red ❌ = issues found
4. **Generate visual tests** - Use `saveBlockHTML()` to inspect in browser

## Important Notes

- This tests **source code** for standards compliance
- Visual testing still requires browser preview
- Multiple instance tests verify block works when duplicated on same page
- Variation tests check that block variations work correctly

**Next:** Run Cell 1 to initialize the environment

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// CELL 1: INITIALIZE ENVIRONMENT (RUN THIS FIRST)
// ═══════════════════════════════════════════════════════════════════

const { JSDOM } = require('jsdom');
const fs = require('fs');
const path = require('path');

// Create test output directory
const testDir = path.join(process.cwd(), 'ipynb-tests');
if (!fs.existsSync(testDir)) {
  fs.mkdirSync(testDir, { recursive: true });
}

// Initialize jsdom
const dom = new JSDOM('<!DOCTYPE html><html><head></head><body></body></html>', {
  url: 'http://localhost:3000',
  pretendToBeVisual: true,
  resources: 'usable'
});

global.window = dom.window;
global.document = dom.window.document;
global.navigator = dom.window.navigator;
global.HTMLElement = dom.window.HTMLElement;

console.log('✅ Environment initialized');
console.log('📁 Test output directory:', testDir);

// ═══════════════════════════════════════════════════════════════════
// HELPER FUNCTIONS
// ═══════════════════════════════════════════════════════════════════

/**
 * Load block styles from CSS file
 */
async function loadBlockStyles(blockName) {
  const cssPath = path.join(process.cwd(), 'blocks', blockName, `${blockName}.css`);
  if (fs.existsSync(cssPath)) {
    return fs.readFileSync(cssPath, 'utf-8');
  }
  return '';
}

/**
 * Test a block by loading its JS and decorating provided HTML
 */
async function testBlock(blockName, innerHTML) {
  // Create block element
  const block = document.createElement('div');
  block.className = `${blockName} block`;
  block.innerHTML = innerHTML;

  // Load and execute block JavaScript
  const jsPath = path.join(process.cwd(), 'blocks', blockName, `${blockName}.js`);
  if (!fs.existsSync(jsPath)) {
    throw new Error(`Block JS not found: ${jsPath}`);
  }

  // Clear require cache and load fresh
  delete require.cache[require.resolve(jsPath)];
  const blockModule = require(jsPath);
  const decorate = blockModule.default || blockModule;

  // Decorate the block
  await decorate(block);

  return block;
}

/**
 * Save block HTML with styles for browser preview
 */
async function saveBlockHTML(blockName, innerHTML, filename) {
  const block = await testBlock(blockName, innerHTML);
  const outputFilename = filename || `${blockName}-preview.html`;
  const outputPath = path.join(testDir, outputFilename);

  // Get relative path to CSS file from test output directory
  const cssRelativePath = `../blocks/${blockName}/${blockName}.css`;

  const html = `<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>${blockName} Test</title>
  <link rel="stylesheet" href="../styles/styles.css">
  <link rel="stylesheet" href="${cssRelativePath}">
  <style>
    body { padding: 2rem; max-width: 1200px; margin: 0 auto; }
    body.appear { display: block; }
  </style>
</head>
<body class="appear">
  <h1>${blockName} Block Test</h1>
  ${block.outerHTML}
</body>
</html>`;

  fs.writeFileSync(outputPath, html);
  console.log(`✅ Preview saved: ${outputPath}`);
  console.log(`   Open in browser to see styled result`);
  console.log(`   Edit ${cssRelativePath} and refresh browser for instant CSS updates!`);

  return outputPath;
}

console.log('✅ Helper functions loaded:');
console.log('   - testBlock(blockName, innerHTML)');
console.log('   - saveBlockHTML(blockName, innerHTML, filename?)');
console.log('   - loadBlockStyles(blockName)');
console.log('');
console.log('Ready to test blocks! 🚀');

---

## Part 1: Standards Compliance Verification

These helper functions check if blocks follow EDS standards.

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// VERIFICATION HELPER: Check for Reserved Class Names
// ═══════════════════════════════════════════════════════════════════

/**
 * Verify block doesn't use reserved class names in CSS
 * Reserved: .{blockname}-container, .{blockname}-wrapper
 */
function verifyNoReservedClasses(blockName) {
  const cssPath = path.join(process.cwd(), 'blocks', blockName, `${blockName}.css`);
  
  if (!fs.existsSync(cssPath)) {
    console.log(`⚠️  No CSS file found for ${blockName}`);
    return true;
  }

  const css = fs.readFileSync(cssPath, 'utf-8');
  const issues = [];

  // Check for .{blockname}-container
  const containerPattern = new RegExp(`\\.${blockName}-container\\s*\\{`, 'g');
  if (containerPattern.test(css)) {
    issues.push(`❌ CRITICAL: Uses reserved class .${blockName}-container`);
  }

  // Check for .{blockname}-wrapper
  const wrapperPattern = new RegExp(`\\.${blockName}-wrapper\\s*\\{`, 'g');
  if (wrapperPattern.test(css)) {
    issues.push(`❌ CRITICAL: Uses reserved class .${blockName}-wrapper`);
  }

  if (issues.length > 0) {
    console.log(`\n🔴 ${blockName}: Reserved Class Name Issues`);
    issues.forEach(issue => console.log(`   ${issue}`));
    console.log(`   Fix: Use -content, -inner, -main, -panel instead`);
    return false;
  }

  console.log(`✅ ${blockName}: No reserved class names`);
  return true;
}

console.log('✅ Loaded: verifyNoReservedClasses(blockName)');

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// VERIFICATION HELPER: Check for Global Selectors
// ═══════════════════════════════════════════════════════════════════

/**
 * Verify block uses block parameter, not document.querySelector()
 * Allows legitimate global selectors (meta, head, main, body)
 */
function verifyNoGlobalSelectors(blockName) {
  const jsPath = path.join(process.cwd(), 'blocks', blockName, `${blockName}.js`);
  
  if (!fs.existsSync(jsPath)) {
    console.log(`⚠️  No JS file found for ${blockName}`);
    return true;
  }

  const js = fs.readFileSync(jsPath, 'utf-8');
  const lines = js.split('\n');
  const issues = [];

  lines.forEach((line, index) => {
    // Skip comments
    if (line.trim().startsWith('//') || line.trim().startsWith('*')) return;

    // Check for document.querySelector
    if (line.includes('document.querySelector')) {
      // Allow legitimate global selectors
      const legitimateSelectors = ['meta[', '"head"', "'head'", '"main"', "'main'", '"body"', "'body'"];
      const isLegitimate = legitimateSelectors.some(selector => line.includes(selector));

      if (!isLegitimate) {
        issues.push(`   Line ${index + 1}: ${line.trim()}`);
      }
    }
  });

  if (issues.length > 0) {
    console.log(`\n🔴 ${blockName}: Global Selector Issues`);
    console.log(`❌ CRITICAL: Found document.querySelector() that should use block parameter:`);
    issues.forEach(issue => console.log(issue));
    console.log(`   Fix: Replace document.querySelector() with block.querySelector()`);
    return false;
  }

  console.log(`✅ ${blockName}: Uses block parameter correctly`);
  return true;
}

console.log('✅ Loaded: verifyNoGlobalSelectors(blockName)');

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// VERIFICATION HELPER: Check for Inline CSS
// ═══════════════════════════════════════════════════════════════════

/**
 * Verify block doesn't use inline CSS (element.style)
 */
function verifyNoInlineCSS(blockName) {
  const jsPath = path.join(process.cwd(), 'blocks', blockName, `${blockName}.js`);
  
  if (!fs.existsSync(jsPath)) {
    console.log(`⚠️  No JS file found for ${blockName}`);
    return true;
  }

  const js = fs.readFileSync(jsPath, 'utf-8');
  const lines = js.split('\n');
  const issues = [];

  lines.forEach((line, index) => {
    // Skip comments
    if (line.trim().startsWith('//') || line.trim().startsWith('*')) return;

    // Check for .style. (but allow .style.cssText)
    if (line.includes('.style.') && !line.includes('.style.cssText')) {
      issues.push(`   Line ${index + 1}: ${line.trim()}`);
    }
  });

  if (issues.length > 0) {
    console.log(`\n🔴 ${blockName}: Inline CSS Issues`);
    console.log(`⚠️  HIGH: Found inline CSS that should use CSS classes:`);
    issues.slice(0, 5).forEach(issue => console.log(issue));
    if (issues.length > 5) {
      console.log(`   ... and ${issues.length - 5} more`);
    }
    console.log(`   Fix: Replace element.style with classList.add/remove`);
    return false;
  }

  console.log(`✅ ${blockName}: No inline CSS`);
  return true;
}

console.log('✅ Loaded: verifyNoInlineCSS(blockName)');

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// VERIFICATION HELPER: Check for CONFIG Object
// ═══════════════════════════════════════════════════════════════════

/**
 * Verify block has a CONFIG object
 */
function verifyHasConfig(blockName) {
  const jsPath = path.join(process.cwd(), 'blocks', blockName, `${blockName}.js`);
  
  if (!fs.existsSync(jsPath)) {
    console.log(`⚠️  No JS file found for ${blockName}`);
    return true;
  }

  const js = fs.readFileSync(jsPath, 'utf-8');
  
  // Check for CONFIG pattern
  const hasConfig = /_CONFIG\s*=/.test(js);

  if (!hasConfig) {
    console.log(`\n🟡 ${blockName}: Missing CONFIG Object`);
    console.log(`⚠️  MEDIUM: No CONFIG object found`);
    console.log(`   Fix: Add BLOCKNAME_CONFIG object with error messages, paths, UI text`);
    return false;
  }

  console.log(`✅ ${blockName}: Has CONFIG object`);
  return true;
}

console.log('✅ Loaded: verifyHasConfig(blockName)');

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// VERIFICATION HELPER: Check for Error Handling
// ═══════════════════════════════════════════════════════════════════

/**
 * Verify async functions have try/catch error handling
 */
function verifyErrorHandling(blockName) {
  const jsPath = path.join(process.cwd(), 'blocks', blockName, `${blockName}.js`);
  
  if (!fs.existsSync(jsPath)) {
    console.log(`⚠️  No JS file found for ${blockName}`);
    return true;
  }

  const js = fs.readFileSync(jsPath, 'utf-8');
  
  const isAsync = /export\s+default\s+async\s+function/.test(js);
  const hasTryCatch = /try\s*\{/.test(js);

  if (isAsync && !hasTryCatch) {
    console.log(`\n🟡 ${blockName}: Missing Error Handling`);
    console.log(`⚠️  MEDIUM: Async function without try/catch`);
    console.log(`   Fix: Add try/catch with user-friendly error message`);
    return false;
  }

  if (isAsync && hasTryCatch) {
    console.log(`✅ ${blockName}: Has error handling`);
    return true;
  }

  console.log(`✅ ${blockName}: Not async (error handling N/A)`);
  return true;
}

console.log('✅ Loaded: verifyErrorHandling(blockName)');

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// COMBINED VERIFICATION: Run All Checks
// ═══════════════════════════════════════════════════════════════════

/**
 * Run all verification checks for a block
 */
function verifyBlockStandards(blockName) {
  console.log(`\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━`);
  console.log(`📋 VERIFYING: ${blockName}`);
  console.log(`━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n`);

  const results = {
    reservedClasses: verifyNoReservedClasses(blockName),
    globalSelectors: verifyNoGlobalSelectors(blockName),
    inlineCSS: verifyNoInlineCSS(blockName),
    config: verifyHasConfig(blockName),
    errorHandling: verifyErrorHandling(blockName)
  };

  const criticalPass = results.reservedClasses && results.globalSelectors;
  const highPass = results.inlineCSS;
  const mediumPass = results.config && results.errorHandling;
  const allPass = criticalPass && highPass && mediumPass;

  console.log(`\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━`);
  if (allPass) {
    console.log(`🎉 ${blockName}: ALL CHECKS PASSED`);
  } else if (criticalPass && highPass) {
    console.log(`✅ ${blockName}: CRITICAL/HIGH checks passed (MEDIUM issues remain)`);
  } else if (criticalPass) {
    console.log(`⚠️  ${blockName}: CRITICAL checks passed (HIGH/MEDIUM issues remain)`);
  } else {
    console.log(`❌ ${blockName}: CRITICAL ISSUES FOUND - FIX IMMEDIATELY`);
  }
  console.log(`━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n`);

  return results;
}

console.log('✅ Loaded: verifyBlockStandards(blockName)');
console.log('');
console.log('🎯 ALL VERIFICATION FUNCTIONS READY!');
console.log('   Use: verifyBlockStandards("blockname") to run all checks');

---

## Part 2: Test Your Fixed Blocks

Use these cells to verify blocks you've fixed follow EDS standards.

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// TEST: Verify a Specific Block
// ═══════════════════════════════════════════════════════════════════

// Change 'showcaser' to the block you want to test
const blockToTest = 'showcaser';

verifyBlockStandards(blockToTest);

: 

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// TEST: Single Instance (Basic Test)
// ═══════════════════════════════════════════════════════════════════

// Test block with single instance to verify basic decoration works
const blockName = 'showcaser'; // Change to block you're testing

const singleContent = `
  <div>
    <div>Title</div>
    <div>Content goes here</div>
  </div>
`;

console.log('🧪 Testing single instance...');

try {
  const block = await testBlock(blockName, singleContent);
  console.log('✅ Single instance test passed');
  console.log('   Decorated HTML length:', block.outerHTML.length, 'chars');
  console.log('   Classes:', block.className);
} catch (error) {
  console.error('❌ Single instance test failed:', error.message);
}

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// TEST: Multiple Instances (Critical for Global Selector Check)
// ═══════════════════════════════════════════════════════════════════

// Test that blocks work correctly when multiple instances on same page
// This catches document.querySelector() bugs

const blockName2 = 'showcaser'; // Change to block you're testing

const content1 = '<div><div>Instance 1 Title</div><div>Content 1</div></div>';
const content2 = '<div><div>Instance 2 Title</div><div>Content 2</div></div>';
const content3 = '<div><div>Instance 3 Title</div><div>Content 3</div></div>';

console.log('🧪 Testing multiple instances...');
console.log('   This test verifies block uses block parameter, not document.querySelector()');

try {
  const block1 = await testBlock(blockName2, content1);
  const block2 = await testBlock(blockName2, content2);
  const block3 = await testBlock(blockName2, content3);

  console.log('\n✅ All instances decorated successfully');
  console.log('   Instance 1:', block1.outerHTML.substring(0, 100) + '...');
  console.log('   Instance 2:', block2.outerHTML.substring(0, 100) + '...');
  console.log('   Instance 3:', block3.outerHTML.substring(0, 100) + '...');
  
  // Verify each instance has different content (proves they're independent)
  const hasInstance1Content = block1.textContent.includes('Instance 1');
  const hasInstance2Content = block2.textContent.includes('Instance 2');
  const hasInstance3Content = block3.textContent.includes('Instance 3');

  if (hasInstance1Content && hasInstance2Content && hasInstance3Content) {
    console.log('\n🎉 PASSED: Each instance has correct content');
    console.log('   Block correctly uses block parameter, not global selectors');
  } else {
    console.error('\n❌ FAILED: Instances have wrong content');
    console.error('   Block likely uses document.querySelector() instead of block parameter');
    console.error('   Instance 1 has "Instance 1":', hasInstance1Content);
    console.error('   Instance 2 has "Instance 2":', hasInstance2Content);
    console.error('   Instance 3 has "Instance 3":', hasInstance3Content);
  }
} catch (error) {
  console.error('❌ Multiple instance test failed:', error.message);
  console.error('   This usually means block has errors when processing multiple instances');
}

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// TEST: Block Variations
// ═══════════════════════════════════════════════════════════════════

// Test block variations (e.g., block with 'bold' or 'large' class)

const blockName3 = 'showcaser'; // Change to block you're testing
const variation = 'featured'; // Change to variation class

const variationContent = '<div><div>Variation Test</div><div>Content</div></div>';

console.log(`🧪 Testing variation: ${blockName3}.${variation}`);

try {
  // Create block with variation class
  const block = document.createElement('div');
  block.className = `${blockName3} ${variation} block`;
  block.innerHTML = variationContent;

  // Load and execute block JavaScript
  const jsPath = path.join(process.cwd(), 'blocks', blockName3, `${blockName3}.js`);
  delete require.cache[require.resolve(jsPath)];
  const blockModule = require(jsPath);
  const decorate = blockModule.default || blockModule;
  await decorate(block);

  console.log('✅ Variation test passed');
  console.log('   Classes:', block.className);
  console.log('   Variation handled correctly');
} catch (error) {
  console.error('❌ Variation test failed:', error.message);
}

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// TEST: Generate Visual Preview
// ═══════════════════════════════════════════════════════════════════

// Generate HTML file with styling for browser inspection

const blockName4 = 'showcaser'; // Change to block you're testing

const visualContent = `
  <div>
    <div>Visual Test Title</div>
    <div>
      <p>This is a visual test of the block.</p>
      <p>Open the generated HTML file in a browser to inspect styling.</p>
    </div>
  </div>
`;

console.log('🎨 Generating visual preview...');

try {
  const outputPath = await saveBlockHTML(blockName4, visualContent);
  console.log('\n✅ Visual preview ready!');
  console.log('   Next steps:');
  console.log('   1. Open file in browser:', outputPath);
  console.log('   2. Inspect styling and layout');
  console.log('   3. Edit blocks/${blockName4}/${blockName4}.css');
  console.log('   4. Refresh browser to see CSS changes instantly!');
} catch (error) {
  console.error('❌ Visual preview failed:', error.message);
}

---

## Part 3: Batch Testing

Test multiple blocks at once to verify Phase 1, 2, or 3 fixes.

In [ ]:
// ═══════════════════════════════════════════════════════════════════
// BATCH TEST: Phase 1 Critical Blocks
// ═══════════════════════════════════════════════════════════════════

const phase1Blocks = [
  'ipynb-viewer',
  'showcaser',
  'blogroll',
  'hero',
  'index',
  'dps',
  'returntotop',
  'floating-alert',
  'dashboard'
];

console.log('🔍 BATCH TEST: Phase 1 Critical Blocks');
console.log('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n');

const phase1Results = {};

for (const blockName of phase1Blocks) {
  const results = verifyBlockStandards(blockName);
  phase1Results[blockName] = results;
}

// Summary
console.log('\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━');
console.log('📊 PHASE 1 SUMMARY');
console.log('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n');

let passedAll = 0;
let passedCritical = 0;
let hasCriticalIssues = 0;

for (const [blockName, results] of Object.entries(phase1Results)) {
  const criticalPass = results.reservedClasses && results.globalSelectors;
  const highPass = results.inlineCSS;
  const mediumPass = results.config && results.errorHandling;
  const allPass = criticalPass && highPass && mediumPass;

  if (allPass) {
    console.log(`✅ ${blockName} - All checks passed`);
    passedAll++;
  } else if (criticalPass && highPass) {
    console.log(`🟡 ${blockName} - CRITICAL/HIGH passed, MEDIUM issues`);
    passedCritical++;
  } else {
    console.log(`❌ ${blockName} - CRITICAL issues found`);
    hasCriticalIssues++;
  }
}

console.log('\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━');
console.log(`Total blocks: ${phase1Blocks.length}`);
console.log(`✅ Passed all: ${passedAll}`);
console.log(`🟡 Passed critical: ${passedCritical}`);
console.log(`❌ Critical issues: ${hasCriticalIssues}`);
console.log('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━');

---

## Usage Examples

### Quick Standards Check
```javascript
verifyBlockStandards('showcaser');
```

### Test Single Instance
```javascript
const block = await testBlock('showcaser', '<div><div>Title</div><div>Content</div></div>');
console.log(block.outerHTML);
```

### Generate Visual Preview
```javascript
await saveBlockHTML('showcaser', '<div><div>Title</div><div>Content</div></div>');
// Open ipynb-tests/showcaser-preview.html in browser
```

### Test Multiple Instances
```javascript
const block1 = await testBlock('showcaser', '<div><div>Block 1</div></div>');
const block2 = await testBlock('showcaser', '<div><div>Block 2</div></div>');
// Verify both work independently
```

---

## Next Steps

1. **Run verification** on blocks you've fixed
2. **Fix any issues** found by the verification functions
3. **Test with multiple instances** to ensure no global selector bugs
4. **Generate visual previews** to confirm styling works
5. **Document fixes** in fix.md file

Ready to test! 🚀